# Using FABnet IPv6 Networks

## Configure the Environment

In [ ]:
import os

# If you are using the FABRIC JupyterHub, the following three evnrionment vars
# were automatically provided when you logged in.
#os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
#os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
#os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

# Bastion IPs
os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'

# Set your Bastion username and private key
os.environ['FABRIC_BASTION_USERNAME']=<INSERT_YOUR_FABRIC_USERNAME>
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/fabric_bastion_key'

# Set the keypair FABRIC will install in your slice. 
os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

# If your slice private key uses a passphrase, set the passphrase
#from getpass import getpass
#print('Please input private key passphrase. Press enter for no passphrase.')
#os.environ['FABRIC_SLICE_PRIVATE_KEY_PASSPHRASE']=getpass()

## Setup the Experiment

#### Import FABRIC API

In [ ]:
import json
import traceback

from fabrictestbed_extensions.fablib.fablib import fablib


In [ ]:
try:
    print(f"{fablib.list_sites()}")
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()


## Create Slice


## Configure Slice Parameters

Set some slice names and other parameters.  Get two different random sites to force wide-area networking. 



In [ ]:
slice_name = 'MySliceL2Net'
[site1,site2] = fablib.get_random_sites(count=2)
print(f"Sites: {site1}, {site2}")

node1_name = 'Node1'
node2_name = 'Node2'

network1_name='net1'
network2_name='net2'

node1_nic_name = 'nic1'
node2_nic_name = 'nic2'

### Create the Slice

NIC component models options:
- NIC_Basic 
- NIC_ConnectX_5 
- NIC_ConnectX_6

In [ ]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Node1
    node1 = slice.add_node(name=node1_name, site=site1)
    iface1 = node1.add_component(model='NIC_Basic', name=node1_nic_name).get_interfaces()[0]
    
    # Node2
    node2 = slice.add_node(name=node2_name, site=site2)
    iface2  = node2.add_component(model='NIC_Basic', name=node2_nic_name).get_interfaces()[0]
    
    # NetworkS
    net1 = slice.add_l3network(name=network1_name, interfaces=[iface1], type='IPv6')
    net2 = slice.add_l3network(name=network2_name, interfaces=[iface2], type='IPv6')
    
    #Submit Slice Request
    slice.submit()
except Exception as e:
    print(f"Exception: {e}")


## Get the Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice}")
except Exception as e:
    print(f"Exception: {e}")


## Get the Nodes

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice.list_nodes()}")
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print(f"{node}")
except Exception as e:
    print(f"Exception: {e}")

## Get the Interfaces

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    print(f"{slice.list_interfaces()}")
except Exception as e:
    print(f"Exception: {e}")

### Get the Networks

The we will use the assigned gateway and subnet to configure the nodes.

In [ ]:
try:
    network1 = slice.get_network(name=network1_name)
    network1_available_ips = network1.get_available_ips()
    print(f"{network1}")
    
    network2 = slice.get_network(name=network2_name)
    network2_available_ips =  network2.get_available_ips()
    print(f"{network2}")
except Exception as e:
    print(f"Exception: {e}")

### Configure Node1

Configure the interface on node1.  

In [ ]:
try:
    node1 = slice.get_node(name=node1_name)        
    node1_iface = node1.get_interface(network_name=network1_name)  
    node1_iface.ip_addr_add(addr=network1_available_ips.pop(0), subnet=network1.get_subnet())
    
    node1.ip_route_add(subnet=network2.get_subnet(), gateway=network1.get_gateway())
    
    stdout, stderr = node1.execute(f'ip -6 addr show {node1_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node1.execute(f'ip -6 route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

### Configure Node2

Configure the interface on on  node2.  

```
ip addr add 192.168.1.11/24 dev eth1
```

In [ ]:
try:
    node2 = slice.get_node(name=node2_name)        
    node2_iface = node2.get_interface(network_name=network2_name)  
    node2_iface.ip_addr_add(addr=network2_available_ips.pop(0), subnet=network2.get_subnet())
    
    node2.ip_route_add(subnet=network1.get_subnet(), gateway=network2.get_gateway())
    
    stdout, stderr = node2.execute(f'ip -6 addr show {node2_iface.get_os_interface()}')
    print (stdout)
    
    stdout, stderr = node2.execute(f'ip -6 route list')
    print (stdout)
except Exception as e:
    print(f"Exception: {e}")
    traceback.print_exc()

## Delete Slice

In [ ]:
try:
    slice = fablib.get_slice(name=slice_name)
    slice.delete()
except Exception as e:
    print(f"Exception: {e}")